In [8]:
import random
import time
import duckdb
import pandas as pd
import sys
import weakref
from pympler.asizeof import asizeof


In [2]:
letters = ['qwertyuiopasdfghjklzxcvbnm']
class Thing:
    def __init__(self):
        self.x = random.random()
        self.y = random.random()
        #self.s = ''.join(random.choice(letters) for _ in range(5))

n_things = 10**7
things = [Thing() for _ in range(n_things)]

# 10^7: 2.3G

In [5]:
# connect to an in-memory database
con = duckdb.connect()

t0 = time.time()
# build a pandas df and insert that; this is ~10x faster than using executemany() to insert a long list of values.
df = pd.DataFrame({
    'x': [t.x for t in things],
    'y': [t.y for t in things],
    'obj_id': [id(t) for t in things],
    }
)

# convert the df into a DuckDB db
con.execute('DROP TABLE IF EXISTS my_table')
t0 = time.time()
con.execute("CREATE TABLE my_table AS SELECT * FROM df")
t1 = time.time()
del df
print(t1-t0)
# Ater: 2.8GB, so 500MB used

0.5390448570251465


In [14]:

t0 = time.time()
obj_lookup = {id(t): t for t in things}
t1 = time.time()
print(t1-t0)
# 3.5GB, so another 600MB there.
# Roughly 1.2GB spent on index total.

3.414109945297241


In [15]:
con.execute('SELECT obj_id FROM my_table LIMIT 10')
res = con.fetchall()
print(res)

[(139762278905264,), (139762278905360,), (139762278905456,), (139762278905552,), (139762278905648,), (139762278905744,), (139762278905840,), (139762278905936,), (139762278906032,), (139762278906128,)]


In [62]:
t0 = time.time()
df = pd.DataFrame({'t': things, 'obj_id': [id(t) for t in things]})
t1 = time.time()
t1-t0

12.824144840240479

In [70]:
t0 = time.time()
for s in sl:
    pass
t1 = time.time()
print(t1-t0)

0.35831308364868164


In [67]:
import ctypes
n_runs = 10
for e in range(0, 8):
    thresh = 10**e / 10**7
    t_duck = 0
    t_ls = 0
    for _ in range(n_runs):
        t0 = time.time()
        con.execute("SELECT obj_id FROM my_table where x <=1 and y <= {} ORDER BY obj_id".format(thresh))
        res = con.fetchall()
        obj_ids = [obj_lookup[r[0]] for r in res]
        objs = df[df.obj_id.isin(obj_ids)].t.to_list()
        #objs = [obj_lookup[r[0]] for r in res]
        #objs = [ctypes.cast(r[0], ctypes.py_object).value for r in res]
        t1 = time.time()
        #objs_ls = [t for t in things if t.x<=1 and t.y <= thresh]
        t2 = time.time()
        t_duck += (t1-t0)/n_runs
        t_ls += (t2-t1) / n_runs
    print(t_duck)

0.2728706121444702
1.5862774133682251
0.7015307903289795
0.6235203504562378
0.9881726026535034


KeyboardInterrupt: 